In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [23]:
# ll = [{'role': 'system', 'content': 'You will get instructions for code to write.\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\nMake sure that every detail of the architecture is, in the end, implemented as code.\n\nThink step by step and reason yourself to the right decisions to make sure we get it right.\nYou will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.\n\nThen you will output the content of each file including ALL code.\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such that\nFILENAME is the lowercase file name including the file extension,\nLANG is the markup code block language for the code\'s language, and CODE is the code:\n\nFILENAME\n```LANG\nCODE\n```\n\nYou will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.\nPlease note that the code should be fully functional. No placeholders.\n\nFollow a language and framework appropriate best practice file naming convention.\nMake sure that files contain all imports, types etc. Make sure that code in different files are compatible with each other.\nEnsure to implement all code, if you are unsure, write a plausible implementation.\nInclude module dependency or package manager dependency definition file.\nBefore you finish, double check that all parts of the architecture is present in the files.\n\nUseful to know:\nYou almost always put different classes in different files.\nFor Python, you always create an appropriate requirements.txt file.\nFor NodeJS, you always create an appropriate package.json file.\nYou always add a comment briefly describing the purpose of the function definition.\nYou try to add comments explaining very complex bits of logic.\nYou always follow the best practices for the requested languages in terms of describing the code written as a defined\npackage/project.\n\n\nPython toolbelt preferences:\n- pytest\n- dataclasses\n'}, {'role': 'user', 'content': 'This is my CSV file:\n```\nyear,period of the year,metric,Total Group,Zone AMS,Zone EMENA,Zone AOA,Nespresso,Nestlé Health Science,Other Businesses\n2020,Three-month,Real internal growth (RIG),4.70,7.90,8.20,-4.60,2.70,18.70,6.80\n2020,Three-month,Pricing,-0.40,-0.50,-1.10,0.00,1.10,-0.20,-2.70\n2020,Three-month,Organic growth,4.30,7.40,7.10,-4.60,3.80,18.50,4.10\n```\n\nWhat can we observe from the data?'}, {'role': 'assistant', 'content': 'Areas that need clarification:\n1. What does "Total Group" refer to?\n2. What do the different zones (AMS, EMENA, AOA) represent?\n3. What is the unit of measurement for the "metric" column?\n4. What does "Real internal growth (RIG)" mean?\n5. What is the time frame for the "Three-month" period?\n6. What do the values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent?'}, {'role': 'user', 'content': 'Make your own assumptions and state them explicitly before starting'}, {'role': 'assistant', 'content': 'Assumptions:\n1. "Total Group" refers to the overall performance of the company.\n2. The different zones (AMS, EMENA, AOA) represent geographical regions or divisions within the company.\n3. The unit of measurement for the "metric" column is percentage (%).\n4. "Real internal growth (RIG)" refers to the growth of the company excluding the impact of external factors such as acquisitions or divestments.\n5. The "Three-month" period refers to a quarterly time frame.\n6. The values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent the growth or performance of specific business units or product categories within the company.'}]
ll = [{'role': 'system', 'content': 'You will get instructions for code to write.\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\nMake sure that every detail of the architecture is, in the end, implemented as code.\n\nThink step by step and reason yourself to the right decisions to make sure we get it right.\nYou will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.\n\nThen you will output the content of each file including ALL code.\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such that\nFILENAME is the lowercase file name including the file extension,\nLANG is the markup code block language for the code\'s language, and CODE is the code:\n\nFILENAME\n```LANG\nCODE\n```\n\nYou will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.\nPlease note that the code should be fully functional. No placeholders.\n\nFollow a language and framework appropriate best practice file naming convention.\nMake sure that files contain all imports, types etc. Make sure that code in different files are compatible with each other.\nEnsure to implement all code, if you are unsure, write a plausible implementation.\nInclude module dependency or package manager dependency definition file.\nBefore you finish, double check that all parts of the architecture is present in the files.\n\nUseful to know:\nYou almost always put different classes in different files.\nFor Python, you always create an appropriate requirements.txt file.\nFor NodeJS, you always create an appropriate package.json file.\nYou always add a comment briefly describing the purpose of the function definition.\nYou try to add comments explaining very complex bits of logic.\nYou always follow the best practices for the requested languages in terms of describing the code written as a defined\npackage/project.\n\n\nPython toolbelt preferences:\n- pytest\n- dataclasses\n'}, {'role': 'user', 'content': 'This is my CSV file:\n```\nyear,period of the year,metric,Total Group,Zone AMS,Zone EMENA,Zone AOA,Nespresso,Nestlé Health Science,Other Businesses\n2020,Three-month,Real internal growth (RIG),4.70,7.90,8.20,-4.60,2.70,18.70,6.80\n2020,Three-month,Pricing,-0.40,-0.50,-1.10,0.00,1.10,-0.20,-2.70\n2020,Three-month,Organic growth,4.30,7.40,7.10,-4.60,3.80,18.50,4.10\n```\n\nWhat can we observe from the data?'}, {'role': 'assistant', 'content': 'Areas that need clarification:\n1. What does "Total Group" refer to?\n2. What do the different zones (AMS, EMENA, AOA) represent?\n3. What is the unit of measurement for the "metric" column?\n4. What does "Real internal growth (RIG)" mean?\n5. What is the time frame for the "Three-month" period?\n6. What do the values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent?'}, {'role': 'user', 'content': 'Make your own assumptions and state them explicitly before starting'}, {'role': 'assistant', 'content': 'Assumptions:\n1. "Total Group" refers to the overall performance of the company.\n2. The different zones (AMS, EMENA, AOA) represent geographical regions or divisions within the company.\n3. The unit of measurement for the "metric" column is percentage (%).\n4. "Real internal growth (RIG)" refers to the growth of the company excluding the impact of external factors such as acquisitions or divestments.\n5. The "Three-month" period refers to a quarterly time frame.\n6. The values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent the growth or performance of specific business units or product categories within the company.'}, {'role': 'user', 'content': 'Please now remember the steps:\n\nThink step by step and reason yourself to the right decisions to make sure we get it right.\nFirst lay out the names of the core classes, functions, methods that will be necessary, As well as a quick comment on their purpose.\n\nThen you will output the content of each file including ALL code.\nEach file must strictly follow a markdown code block format, where the following tokens must be replaced such that\nFILENAME is the lowercase file name including the file extension,\nLANG is the markup code block language for the code\'s language, and CODE is the code:\n\nFILENAME\n```LANG\nCODE\n```\n\nPlease note that the code should be fully functional. No placeholders.\n\nYou will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.\nFollow a language and framework appropriate best practice file naming convention.\nMake sure that files contain all imports, types etc. The code should be fully functional. Make sure that code in different files are compatible with each other.\nBefore you finish, double check that all parts of the architecture is present in the files.\n'}, {'role': 'assistant', 'content': "Based on the requirements, here are the core classes, functions, and methods that will be necessary:\n\n1. CSVParser: A class responsible for parsing the CSV file and extracting the data.\n   - parse_csv_file(file_path: str) -> List[Dict[str, Union[str, float]]]: Parses the CSV file and returns a list of dictionaries representing each row of data.\n\n2. DataAnalyzer: A class responsible for analyzing the data extracted from the CSV file.\n   - calculate_average_growth(data: List[Dict[str, Union[str, float]]]) -> float: Calculates the average growth across all periods.\n   - calculate_zone_growth(data: List[Dict[str, Union[str, float]]], zone: str) -> float: Calculates the growth for a specific zone.\n   - calculate_metric_growth(data: List[Dict[str, Union[str, float]]], metric: str) -> float: Calculates the growth for a specific metric.\n\n3. Main: The entry point of the program.\n   - main(): Reads the CSV file, parses the data, and performs analysis on the data.\n\nNow, let's proceed with implementing the code."}]

print(ll[0]['role'])
print(ll[0]['content'])

system
You will get instructions for code to write.
You will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.
Make sure that every detail of the architecture is, in the end, implemented as code.

Think step by step and reason yourself to the right decisions to make sure we get it right.
You will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.

Then you will output the content of each file including ALL code.
Each file must strictly follow a markdown code block format, where the following tokens must be replaced such that
FILENAME is the lowercase file name including the file extension,
LANG is the markup code block language for the code's language, and CODE is the code:

FILENAME
```LANG
CODE
```

You will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.
Please note that the code should be full

In [17]:
print(ll[1]['role'])
print(ll[1]['content'])

user
This is my CSV file:
```
year,period of the year,metric,Total Group,Zone AMS,Zone EMENA,Zone AOA,Nespresso,Nestlé Health Science,Other Businesses
2020,Three-month,Real internal growth (RIG),4.70,7.90,8.20,-4.60,2.70,18.70,6.80
2020,Three-month,Pricing,-0.40,-0.50,-1.10,0.00,1.10,-0.20,-2.70
2020,Three-month,Organic growth,4.30,7.40,7.10,-4.60,3.80,18.50,4.10
```

What can we observe from the data?


In [18]:
print(ll[2]['role'])
print(ll[2]['content'])

assistant
Areas that need clarification:
1. What does "Total Group" refer to?
2. What do the different zones (AMS, EMENA, AOA) represent?
3. What is the unit of measurement for the "metric" column?
4. What does "Real internal growth (RIG)" mean?
5. What is the time frame for the "Three-month" period?
6. What do the values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent?


In [19]:
print(ll[3]['role'])
print(ll[3]['content'])

user
Make your own assumptions and state them explicitly before starting


In [20]:
print(ll[4]['role'])
print(ll[4]['content'])

assistant
Assumptions:
1. "Total Group" refers to the overall performance of the company.
2. The different zones (AMS, EMENA, AOA) represent geographical regions or divisions within the company.
3. The unit of measurement for the "metric" column is percentage (%).
4. "Real internal growth (RIG)" refers to the growth of the company excluding the impact of external factors such as acquisitions or divestments.
5. The "Three-month" period refers to a quarterly time frame.
6. The values in the "Nespresso", "Nestlé Health Science", and "Other Businesses" columns represent the growth or performance of specific business units or product categories within the company.


In [21]:
# AFTER ai.next
print(ll[5]['role'])
print(ll[5]['content'])

user
Please now remember the steps:

Think step by step and reason yourself to the right decisions to make sure we get it right.
First lay out the names of the core classes, functions, methods that will be necessary, As well as a quick comment on their purpose.

Then you will output the content of each file including ALL code.
Each file must strictly follow a markdown code block format, where the following tokens must be replaced such that
FILENAME is the lowercase file name including the file extension,
LANG is the markup code block language for the code's language, and CODE is the code:

FILENAME
```LANG
CODE
```

Please note that the code should be fully functional. No placeholders.

You will start with the "entrypoint" file, then go to the ones that are imported by that file, and so on.
Follow a language and framework appropriate best practice file naming convention.
Make sure that files contain all imports, types etc. The code should be fully functional. Make sure that code in dif

In [22]:
# AFTER ai.next
print(ll[6]['role'])
print(ll[6]['content'])

assistant
Based on the requirements, here are the core classes, functions, and methods that will be necessary:

1. CSVParser: A class responsible for parsing the CSV file and extracting the data.
   - parse_csv_file(file_path: str) -> List[Dict[str, Union[str, float]]]: Parses the CSV file and returns a list of dictionaries representing each row of data.

2. DataAnalyzer: A class responsible for analyzing the data extracted from the CSV file.
   - calculate_average_growth(data: List[Dict[str, Union[str, float]]]) -> float: Calculates the average growth across all periods.
   - calculate_zone_growth(data: List[Dict[str, Union[str, float]]], zone: str) -> float: Calculates the growth for a specific zone.
   - calculate_metric_growth(data: List[Dict[str, Union[str, float]]], metric: str) -> float: Calculates the growth for a specific metric.

3. Main: The entry point of the program.
   - main(): Reads the CSV file, parses the data, and performs analysis on the data.

Now, let's proceed wi

# Data

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

load_dataset(huggingface_dataset_name)

# Model

pre-trained [**LLM FLAN-T5**](https://huggingface.co/docs/transformers/model_doc/flan-t5) from Hugging Face

In [6]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
# Tokenizer:
# CONVERT RAW TEXT INTO OUR VECTOR SPACE THAT CAN THEN BE PROCESSED BY OUR MODEL
tokenizer = AutoTokenizer.from_pretrained(model_name)